In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://cyxstorage1.blob.core.windows.net/newcontainer/insurance.csv')
df

age     sex     bmi  children smoker     region      charges
0      19  female  27.900         0    yes  southwest  16884.92400
1      18    male  33.770         1     no  southeast   1725.55230
2      28    male  33.000         3     no  southeast   4449.46200
3      33    male  22.705         0     no  northwest  21984.47061
4      32    male  28.880         0     no  northwest   3866.85520
...   ...     ...     ...       ...    ...        ...          ...
1333   50    male  30.970         3     no  northwest  10600.54830
1334   18  female  31.920         0     no  northeast   2205.98080
1335   18  female  36.850         0     no  southeast   1629.83350
1336   21  female  25.800         0     no  southwest   2007.94500
1337   61  female  29.070         0    yes  northwest  29141.36030

[1338 rows x 7 columns]

## Change to categorical values

In [3]:
def set_age(row):
    age = row['age']
    if age <= 20:
        return 1
    elif age <= 30:
        return 2
    elif age <= 40:
        return 3
    elif age <= 50:
        return 4
    elif age <= 60:
        return 5
    else:
        return 6
    
df['age_class'] = df.apply(set_age, axis=1)
df

age     sex     bmi  children smoker     region      charges  age_class
0      19  female  27.900         0    yes  southwest  16884.92400          1
1      18    male  33.770         1     no  southeast   1725.55230          1
2      28    male  33.000         3     no  southeast   4449.46200          2
3      33    male  22.705         0     no  northwest  21984.47061          3
4      32    male  28.880         0     no  northwest   3866.85520          3
...   ...     ...     ...       ...    ...        ...          ...        ...
1333   50    male  30.970         3     no  northwest  10600.54830          4
1334   18  female  31.920         0     no  northeast   2205.98080          1
1335   18  female  36.850         0     no  southeast   1629.83350          1
1336   21  female  25.800         0     no  southwest   2007.94500          2
1337   61  female  29.070         0    yes  northwest  29141.36030          6

[1338 rows x 8 columns]

## Change to dummy variables

In [4]:
dmdf = pd.get_dummies(df, columns=['age_class', 'sex', 'children', 'smoker', 'region'], drop_first=True)
dmdf = dmdf.drop(['age'], axis=1)
columns = list(dmdf.columns.values)
columns[0], columns[1] = columns[1], columns[0]
dmdf = dmdf[columns]
dmdf

charges     bmi  age_class_2  age_class_3  age_class_4  age_class_5  \
0     16884.92400  27.900            0            0            0            0   
1      1725.55230  33.770            0            0            0            0   
2      4449.46200  33.000            1            0            0            0   
3     21984.47061  22.705            0            1            0            0   
4      3866.85520  28.880            0            1            0            0   
...           ...     ...          ...          ...          ...          ...   
1333  10600.54830  30.970            0            0            1            0   
1334   2205.98080  31.920            0            0            0            0   
1335   1629.83350  36.850            0            0            0            0   
1336   2007.94500  25.800            1            0            0            0   
1337  29141.36030  29.070            0            0            0            0   

      age_class_6  sex_male  children_1  children_2  children_3  children_4  \
0               0         0           0           0           0           0   
1               0         1           1           0           0           0   
2               0         1           0           0           1           0   
3               0         1           0           0           0           0   
4               0         1           0           0           0           0   
...           ...       ...         ...         ...         ...         ...   
1333            0         1           0           0           1           0   
1334            0         0           0           0           0           0   
1335            0         0           0           0           0           0   
1336            0         0           0           0           0           0   
1337            1         0           0           0           0           0   

      children_5  smoker_yes  region_northwest  region_southeast  \
0              0           1                 0                 0   
1              0           0                 0                 1   
2              0           0                 0                 1   
3              0           0                 1                 0   
4              0           0                 1                 0   
...          ...         ...               ...               ...   
1333           0           0                 1                 0   
1334           0           0                 0                 0   
1335           0           0                 0                 1   
1336           0           0                 0                 0   
1337           0           1                 1                 0   

      region_southwest  
0                    1  
1                    0  
2                    0  
3                    0  
4                    0  
...                ...  
1333                 0  
1334                 0  
1335                 0  
1336                 1  
1337                 0  

[1338 rows x 17 columns]

## Train models

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split

In [6]:
train, test = train_test_split(dmdf, test_size=0.2)
X_train, y_train = train.loc[:, 'bmi':'region_southwest'], train.loc[:, 'charges']
X_test, y_test = test.loc[:, 'bmi':'region_southwest'], test.loc[:, 'charges']

In [7]:
from sklearn.linear_model import LinearRegression

lreg = LinearRegression()
lreg.fit(X_train, y_train)
pred = lreg.predict(X_test)
r2 = lreg.score(X_train, y_train)
print('R-squared =', r2)

R-squared = 0.7568901398081362


In [8]:
import statsmodels.api as sm

X_train_sm = sm.add_constant(X_train)
sts = sm.OLS(y_train, X_train_sm).fit()
sts.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.757
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     204.9
Date:                Sat, 13 Aug 2022   Prob (F-statistic):          1.41e-309
Time:                        14:08:42   Log-Likelihood:                -10840.
No. Observations:                1070   AIC:                         2.171e+04
Df Residuals:                    1053   BIC:                         2.180e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -6572.9091   1165.102     -5.641      0.000   -8859.094   -4286.724
bmi                345.6337     33.251     10.395      0.000     280.387     410.880
age_class_2       1609.9208    692.980      2.323      0.020     250.141    2969.700
age_class_3       2779.3768    726.827      3.824      0.000    1353.183    4205.571
age_class_4       5880.4049    707.304      8.314      0.000    4492.519    7268.291
age_class_5       9579.8418    699.814     13.689      0.000    8206.652     1.1e+04
age_class_6       1.111e+04    896.880     12.389      0.000    9351.232    1.29e+04
sex_male          -464.2336    377.087     -1.231      0.219   -1204.162     275.695
children_1         581.6785    495.498      1.174      0.241    -390.597    1553.954
children_2        1963.2615    553.651      3.546      0.000     876.876    3049.647
children_3        1213.8250    627.996      1.933      0.054     -18.441    2446.091
children_4        3102.6172   1375.164      2.256      0.024     404.244    5800.991
children_5        2364.9241   1741.203      1.358      0.175   -1051.698    5781.546
smoker_yes        2.406e+04    465.044     51.734      0.000    2.31e+04     2.5e+04
region_northwest  -784.5614    543.203     -1.444      0.149   -1850.445     281.323
region_southeast -1414.0956    544.743     -2.596      0.010   -2483.001    -345.191
region_southwest -1057.4641    546.324     -1.936      0.053   -2129.473      14.544
==============================================================================
Omnibus:                      216.333   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              487.203
Skew:                           1.110   Prob(JB):                    1.60e-106
Kurtosis:                       5.449   Cond. No.                         293.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Results (convert to onnx)

Since the R-squared value is high enough and I am not going to train other models such as random forest or ddep learning networks.

In [9]:
from skl2onnx import to_onnx

onx = to_onnx(lreg, X_train.loc[0, :].to_numpy().astype(np.float32))
with open('lreg.onnx', 'wb') as f:
    f.write(onx.SerializeToString())